<a href="https://colab.research.google.com/github/minhaz1172/NLP/blob/main/Text_Classification(word2vec_andML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


# upload kaggle json

In [2]:
from google.colab import files
files.upload()  # Upload the downloaded kaggle.json file here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


# download and unzip

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip -d data


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 874MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: data/IMDB Dataset.csv   


In [4]:
df=pd.read_csv('/content/data/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [6]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


# separate 1st 20k rows for my project

In [7]:
#df=df.iloc[:20000]
#df.shape

(20000, 2)

## check duplicated

In [12]:
df.duplicated().value_counts()

,count
False,19926
True,74


In [13]:
df.drop_duplicates(inplace=True)

In [15]:
df.duplicated().value_counts()

,count
False,19926
